In [1]:
import sys
sys.path.append('../data/cifar100/')  
from cifar100_loader import load_cifar100

In [3]:
trainloader, validloader, testloader = load_cifar100(batch_size=32, validation_split=0.1)


100.0%


Extracting ./data\cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
print("Dimension of the training dataset:", len(trainloader.dataset))
print("Dimension of the validation dataset:", len(validloader.dataset))
print("Dimension of the test dataset:", len(testloader.dataset))


Dimension of the training dataset: 45000
Dimension of the validation dataset: 5000
Dimension of the test dataset: 10000
